In [1]:
%matplotlib inline  
import mxnet as mx
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt

from vision.cnn import resnet

In [2]:
data_dir = '/media/tx-deepocean/Data/dataset/cifar10/cifar-10-batches-py'

train_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file = 'test_batch'

In [3]:
data = []
label = []

for train_file in train_files:
    with open(os.path.join(data_dir, train_file), 'rb') as fo:
        tmp = pickle.load(fo, encoding='bytes')
        data.append(np.reshape(tmp[b'data'], (tmp[b'data'].shape[0], 3, 32, 32)))
        label.extend(tmp[b'labels'])

In [4]:
data = np.concatenate(data, axis=0)

In [5]:
data.shape

(50000, 3, 32, 32)

In [6]:
test_data = []
test_label = []

with open(os.path.join(data_dir, test_file), 'rb') as fo:
        tmp = pickle.load(fo, encoding='bytes')
        test_data = np.reshape(tmp[b'data'], (tmp[b'data'].shape[0], 3, 32, 32))
        test_label = tmp[b'labels']

In [20]:
batch_size = 500
oh_label = mx.nd.one_hot(mx.nd.array(label), 10)
train_iter = mx.io.NDArrayIter(data, oh_label, batch_size=batch_size, shuffle=True)
oh_label = mx.nd.one_hot(mx.nd.array(test_label), 10)
val_iter = mx.io.NDArrayIter(test_data,oh_label, batch_size=batch_size)

In [21]:
r18 = resnet.ResnetV1(resnet.residual_block_v1, [2, 2, 2, 2], [64, 64, 128, 256, 512], classes=10)

r18_sym = mx.sym.SoftmaxOutput(data=r18.output, name='softmax')

In [22]:
r18_model = mx.mod.Module(symbol=r18_sym, context=[mx.gpu(0),mx.gpu(1)])

In [23]:
import logging
logging.getLogger().setLevel(logging.DEBUG)
r18_model.fit(train_data=train_iter, eval_data=val_iter, optimizer='sgd', 
              optimizer_params={'learning_rate': 0.0001}, eval_metric='acc', 
              batch_end_callback = mx.callback.Speedometer(batch_size, 50), num_epoch=3)

INFO:root:Epoch[0] Batch [50]	Speed: 1686.28 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [100]	Speed: 1691.39 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [150]	Speed: 1694.32 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [200]	Speed: 1675.59 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [250]	Speed: 1674.95 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [300]	Speed: 1682.42 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [350]	Speed: 1685.53 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [400]	Speed: 1677.96 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [450]	Speed: 1699.39 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [500]	Speed: 1679.60 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [550]	Speed: 1676.52 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [600]	Speed: 1684.19 samples/sec	accuracy=0.900000
INFO:root:Epoch[0] Batch [650]	Speed: 1683.93 samples/sec	accuracy=0.900000
INFO:root:Epo